<a href="https://colab.research.google.com/github/shivakumarsanugula/Kaggle-NLP/blob/main/introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#check for GPU

!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [3]:
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


In [4]:
# Get helper functions
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2023-12-04 13:14:31--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-12-04 13:14:31 (63.8 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [5]:
## Get a text dataset

!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

#unzip data
unzip_data("nlp_getting_started.zip")

--2023-12-04 13:14:35--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.1.207, 108.177.121.207, 172.217.214.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.1.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2023-12-04 13:14:36 (101 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [6]:
## Visualizing a text dataset
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
train_df['text'][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [8]:
#shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
# what does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [10]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [11]:
# How many total samples?
len(train_df),len(test_df)

(7613, 3263)

In [12]:
#Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df) -5) # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target:{target}","(real diaster)" if target > 0 else "(not real diaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target:1 (real diaster)
Text:
@DoctorFluxx @StefanEJones @spinnellii @themermacorn  No burning buildings and rob during a riot. That's embarrassing &amp; ruining this nation.

---

Target:0 (not real diaster)
Text:
Why Some Traffic Is Freezing Cold And Some Blazing Hot ÛÒ And How To Heat Up Some Of Your Traffic http://t.co/C8b6DdiQIg

---

Target:0 (not real diaster)
Text:
Video Captures Man Removing American Flag From Long Beach CA Home Burning It; Arsonist Sought http://t.co/JP2QlrunjJ http://t.co/jbpgkGOwSi

---

Target:0 (not real diaster)
Text:
This is my jam: Riser by Dierks Bentley @1061TheTwister ? #iHeartRadio #NowPlaying http://t.co/zQoScQD64h http://t.co/yLvVF139BB

---

Target:0 (not real diaster)
Text:
Ways so archetype a bleeding well-grounded readiness: FpOJ http://t.co/WXbrArc7p3

---



In [13]:
#Splitting data into training and validation sets

from sklearn.model_selection import train_test_split

# use train_test_split to split training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,random_state=42)

In [14]:
# check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [15]:
# check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

In [16]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [17]:
# converting text data to numbers using tokenisation and embeddings

# Text vectorization(tokenization)
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [18]:
# use the defalut TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=10000,# how many words in the vocabulary (automatically add <00v>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None,# create group of n-words
                                    output_mode="int", #how to map tokens to numbers
                                    output_sequence_length=None, # how long do you want your sequences to be
                                    pad_to_max_tokens=True)

In [19]:
train_sentences[0].split()

['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet']

In [20]:
len(train_sentences[0].split())

7

In [21]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [22]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary it is hyper-parameter
max_length = 15 # max length our sequences will be (ex:- how many words from a Tweet does a model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [23]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [24]:
# Create a sample sentence and tokenize it

sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [25]:
# choose a random sentence from the training data and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
        \n\nVectorized version: ")
text_vectorizer([random_sentence])

Original text:
 ÛÏ@FDNY: This morning #FDNY responded to a sinkhole in #Brooklyn. Units remain on-scene with @NYCBuildings &amp; others. http://t.co/M78ir0IK01Û        

Vectorized version: 


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[6452,   19, 1004,    1, 3475,    5,    3,  364,    4, 1319, 1574,
        2321,    1,   14,    1]])>

In [26]:
# Get the unique words in the vocublary
words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words in our training data
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words:{bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words:['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [27]:
#[UNK] unkowun token

# Creating an Embedding using an Embedding Layer
'''
To make our embedding, we're going to use TensorFlow's embedding layer :
https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

The parameters we care most about for our embedding layer:

* input_dim = the size of the vocabulary
* output_dim = the size the output embedded vector, for example, a value would mean each token gets represented by a vector 100 long
* input_length = length of the sequences being passed to the embedding layer

'''

In [28]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # output shape works well which is divisible by 8
                             input_length=max_length # how long is each input
                             )

In [29]:
embedding

In [30]:
# Get a random setence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
       \n\nEmbedded version: ")

# Embed the random sentence (turn into  dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 late night mcdonalds with friends = hilarious although my car is wrecked and there's half a steak pastie in the industrial estate       

Embedded version: 


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.02986386, -0.02174386,  0.01561731, ...,  0.0163266 ,
         -0.02569265, -0.01856427],
        [ 0.04904319, -0.02175288, -0.01751244, ..., -0.02375367,
          0.00128815, -0.04230431],
        [-0.03727277,  0.0493311 , -0.00358015, ..., -0.00994339,
         -0.04020175, -0.01604908],
        ...,
        [ 0.04421004, -0.04797354, -0.00899442, ...,  0.04609858,
         -0.02163379,  0.02319893],
        [ 0.02512406, -0.04443704,  0.02859831, ...,  0.03915682,
          0.01137405, -0.01061384],
        [-0.01626255, -0.00386649,  0.03075321, ..., -0.02254418,
         -0.03037242,  0.03592705]]], dtype=float32)>

In [31]:
# check out a single token's embedding
sample_embed[0][0],sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.02986386, -0.02174386,  0.01561731,  0.01890147,  0.01114614,
         0.04757284, -0.02825662, -0.02732486, -0.0365908 , -0.03776542,
        -0.02630762, -0.02730593, -0.00243003,  0.03224095, -0.02113633,
         0.03884835,  0.01250643,  0.01989495, -0.0057173 , -0.01252406,
         0.00698751, -0.02565062,  0.03615921,  0.03741692,  0.0285542 ,
        -0.00633277,  0.03727001,  0.04455512,  0.01802987, -0.04055388,
         0.03665412, -0.02630202,  0.00296165,  0.00234562, -0.04603552,
         0.02718681,  0.0315344 , -0.03793148,  0.0399881 , -0.01048317,
         0.04858227, -0.01102718, -0.02203096,  0.02064948,  0.0484027 ,
         0.02217828, -0.01857177,  0.04330932, -0.02582324, -0.00531077,
        -0.00442807,  0.0257091 ,  0.03529756, -0.04827749, -0.03461165,
        -0.02004964, -0.04171547,  0.0331012 ,  0.01785122, -0.03727648,
        -0.01732634, -0.04516527,  0.019138  ,  0.04122609,  0.02736641,
  

# Modelling a text dataset (running a series of experiments)

Now we've got way to turn our text sequence into numbers, it's time to start building a series of modelling experiments.

* Model 0 : Naive Bayes (baseline), this is from sklearn ML map: https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html
* Model 1 : Feed-forward neural network (dense model)
* Model 2 : LSTM model (RNN)
* Model 3 : GRU model (RNN)
* Model 4 : Bidirectional- LSTM model (RNN)
* Model 5 : 1D Convolutional Neural Network (CNN)
* Model 6 : TensorFlow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* Model 7 :  same as model 6 with 10% of training data

How are we going to apporach all of these?

Use the standard steps in modelling with tensorflow:
* Create a model
* Build a model
* Fit a model
* Evaluate our model


#Model 0 : Getting a baseline

In [32]:
# Model 0 : Getting a Baseline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline

model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tf-idf
                    ("clf", MultinomialNB()) # model the text
])

#Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [33]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of : {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of : 79.27%


In [34]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [35]:
## Creating an evalution function for our model experiments
# For deepview of metrices: https://scikit-learn.org/stable/modules/model_evaluation.html

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculaute_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary calssification model.
  """
  #Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  #Calculate model precision, recall, and f1-score using "weighted average"
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy" : model_accuracy,
                   "precision" : model_precision,
                   "recall" : model_recall,
                   "f1": model_f1}
  return model_results

In [36]:
# Get baseline results
baseline_results = calculaute_results(y_true=val_labels,
                                      y_pred=baseline_preds)

baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

#Model 1 : Feed-forward neural network (dense model)

In [37]:
# A simple dense model

# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [38]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype=tf.string) # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x) # Create the output layer, want binary outputs so use sigmoid
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")  # construct the model


In [39]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [40]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Fit the model
history_1 = model_1.fit(train_sentences,
                        train_labels,
                        epochs=5,
                        validation_data=(val_sentences, val_labels),
                        callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                               experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20231204-131438
Epoch 1/5
215/215 [==============================] - 6s 24ms/step - loss: 0.6121 - accuracy: 0.6898 - val_loss: 0.5362 - val_accuracy: 0.7572
Epoch 2/5
215/215 [==============================] - 5s 21ms/step - loss: 0.4419 - accuracy: 0.8184 - val_loss: 0.4732 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 4s 20ms/step - loss: 0.3480 - accuracy: 0.8589 - val_loss: 0.4603 - val_accuracy: 0.7887
Epoch 4/5
215/215 [==============================] - 6s 30ms/step - loss: 0.2853 - accuracy: 0.8905 - val_loss: 0.4612 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 5s 24ms/step - loss: 0.2391 - accuracy: 0.9121 - val_loss: 0.4766 - val_accuracy: 0.7874


In [41]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 2ms/step - loss: 0.4766 - accuracy: 0.7874


[0.47655341029167175, 0.787401556968689]

In [42]:
# Make some predictions and evalaute those

model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 2ms/step


(762, 1)

In [43]:
# look at a single prediction
model_1_pred_probs[0]

array([0.39206532], dtype=float32)

In [44]:
# look at a first 10 prediction
model_1_pred_probs[:10]

array([[0.39206532],
       [0.65838736],
       [0.9975396 ],
       [0.09182374],
       [0.11279622],
       [0.9314146 ],
       [0.92000353],
       [0.98393023],
       [0.9595063 ],
       [0.19447215]], dtype=float32)

In [45]:
# Convert model prediction probabilites to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [46]:
# Calculate our model_1 results
model_1_results = calculaute_results(y_true=val_labels,
                                     y_pred=model_1_preds)
model_1_results

{'accuracy': 78.74015748031496,
 'precision': 0.7927656871284042,
 'recall': 0.7874015748031497,
 'f1': 0.7842639469124086}

In [47]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [48]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

In [49]:
## Visualizing learned embedding

# Get the vocabulary from the text vectorization layer

words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [50]:
# Model 1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [52]:
# Get the weight matrix of embedding layer
# (these are the numerical representations of each token in our trainingdata, which have been learned for ~5 epochs)
embed_weights = model_1.get_layer("embedding").get_weights()[0]
print(embed_weights.shape) # same size as vocab size and embedding_dim (output_dim of our embedding layer)

(10000, 128)


Now we've got the embedding matrix our model has learned to represent our tokens, let's see how we can visualize it.

To do so, TensorFlow has a handy tool called projector : https://projector.tensorflow.org/

And Tensorflow also has an incredible guide on word embeding themselves : https://www.tensorflow.org/text/guide/word_embeddings

In [53]:
embed_weights

array([[ 0.05027614,  0.0614404 ,  0.0421002 , ...,  0.06588507,
        -0.05747323, -0.05741385],
       [ 0.00785332,  0.04681252,  0.02741066, ...,  0.04810449,
         0.0220097 ,  0.02932264],
       [ 0.02149481,  0.03820723, -0.05382413, ..., -0.00478615,
        -0.00115613, -0.04913468],
       ...,
       [ 0.0369615 , -0.03421994,  0.0223414 , ...,  0.04766854,
        -0.02115398,  0.02494823],
       [ 0.07956015,  0.06087262,  0.02978045, ...,  0.08757222,
        -0.04285054, -0.07502095],
       [ 0.02645983,  0.06732737,  0.10217768, ...,  0.03111892,
        -0.01808613, -0.03460234]], dtype=float32)

In [54]:
# Creating embedding files (we got this from Tensorflow's word embeddings documentation)
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [55]:
# Download files from colab to upload to projector
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Recurrent Neural Network (RNN)

RNN's are useful for sequence data.

The premise of a recurrent neural network is to use the representation of a previous input to aid the representation of a later input.

LSTM : https://colah.github.io/posts/2015-08-Understanding-LSTMs/



## Model 2 : LSTM
LSTM = long short term memory (one of the most opular LSTM cells)

Our structure of an RNN typically looks like this:

Input(text) -> Tokenize -> Embedding -> Layers (RNNs/Dense) -> Output(label probability)

In [58]:
# Create an LSTM model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x = layers.LSTM(64, return_sequences=True)(x)
print(x.shape)
x = layers.LSTM(64)(x)
print(x.shape)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1,activation='sigmoid')(x)
model_2 = tf.keras.Model(inputs,outputs,name="model_2_LSTM")

(None, 15, 128)
(None, 15, 64)
(None, 64)
